In [ ]:
import requests
import zipfile
import io
import os
import json

# URL du fichier ZIP
url = "https://opendata.paris.fr/api/datasets/1.0/comptages-routiers-permanents-historique/attachments/opendata_txt_2020_zip/"

# Nom local du fichier ZIP
zip_filename = "comptages_routiers_2020.zip"

# Dossier où extraire les fichiers
output_folder = "extracted_data"

# Étape 1 : Télécharger le fichier ZIP
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(zip_filename, "wb") as f:
        f.write(response.content)
    print(f"Fichier téléchargé : {zip_filename}")
else:
    print(f"Erreur lors du téléchargement : {response.status_code}")
    exit()

# Étape 2 : Extraire le contenu du fichier ZIP
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(output_folder)
    print(f"Fichiers extraits dans le dossier : {output_folder}")

# Étape 3 : Lister les fichiers extraits
files = os.listdir(output_folder)
print(f"Fichiers extraits : {files}")

# Étape 4 : Charger le fichier JSON (si présent)
json_files = [file for file in files if file.endswith('.json')]
if json_files:
    json_path = os.path.join(output_folder, json_files[0])  # Charger le premier fichier JSON trouvé
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print("Données chargées avec succès.")
    print(f"Exemple de données : {json.dumps(data[:1], indent=4)}")  # Afficher un exemple
else:
    print("Aucun fichier JSON trouvé dans le ZIP.")
